<a href="https://colab.research.google.com/github/someonesphantom/Music-Gen/blob/main/sheetMusicToMusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install music21
!pip install onnxruntime


In [2]:
from google.colab import files
uploaded = files.upload()


Saving 01.Twinkle_Twinkle_LittleStar_Page_1_707f2ff7-8eb8-47a4-b055-bea4fc58ff84.png to 01.Twinkle_Twinkle_LittleStar_Page_1_707f2ff7-8eb8-47a4-b055-bea4fc58ff84.png


In [3]:
!git clone https://github.com/BreezeWhite/oemer.git
#!pip install -r requirements.txt


fatal: destination path 'oemer' already exists and is not an empty directory.


In [4]:
uploaded_image_path = "/content/" + list(uploaded.keys())[0]
#escaped_image_path = uploaded_image_path.replace("(", "\(").replace(")", "\)")
#print(escaped_image_path)
#/content/01.Twinkle_Twinkle_LittleStar_Page_1_707f2ff7-8eb8-47a4-b055-bea4fc58ff84.png

In [5]:
!python /content/oemer/main.py "{uploaded_image_path}" -o /content/output.musicxml

2025-04-25 09:24:01 Extracting staffline and symbols
/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
1685 2181
2025-04-25 09:29:11 Extracting layers of different symbols
1685 2181
2025-04-25 09:40:05 Dewarping
2025-04-25 09:40:10 Extracting stafflines
95 281 5
282 468 5
469 655 5
656 842 5
843 1029 5
1030 1216 5
1217 1403 5
1404 1590 5
2025-04-25 09:40:11 Extracting noteheads
2025-04-25 09:40:11 Analyzing notehead bboxes
2025-04-25 09:40:11 Instanitiating notes
2025-04-25 09:40:11 Grouping noteheads
2025-04-25 09:40:13 Extracting symbols
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at you

In [6]:
from music21 import converter, note, chord

import json

filename = list(uploaded.keys())[0]
score = converter.parse(filename)

# Extract notes, start times, durations
note_events = []
current_time = 0.0

for el in score.flat.notes:
    dur = float(el.quarterLength)
    if isinstance(el, note.Note):
        note_events.append({
            "pitch": el.nameWithOctave,
            "start_time": current_time,
            "duration": dur
        })
    elif isinstance(el, chord.Chord):
        for n in el.notes:
            note_events.append({
                "pitch": n.nameWithOctave,
                "start_time": current_time,
                "duration": dur
            })
    current_time += dur

note_json = json.dumps(note_events)


ConverterFileException: cannot find a format extensions for: /content/01.Twinkle_Twinkle_LittleStar_Page_1_707f2ff7-8eb8-47a4-b055-bea4fc58ff84.png

In [ ]:
from IPython.display import HTML, display

display(HTML(f"""
<iframe width="100%" height="450" srcdoc='
<html>
<head>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/tone/14.8.39/Tone.js"></script>
  <style>
    body {{ font-family: sans-serif; }}
    .piano {{ display: flex; flex-wrap: wrap; max-width: 1000px; margin: 10px auto; }}
    .key {{
      width: 40px;
      height: 150px;
      border: 1px solid #000;
      box-sizing: border-box;
      text-align: center;
      position: relative;
      background: white;
      color: black;
      font-size: 10px;
    }}
    .black {{ background: black; color: white; height: 100px; width: 30px; margin-left: -15px; margin-right: -15px; z-index: 2; }}
    .active {{ background: yellow !important; color: black !important; }}
    .label {{ position: absolute; top: -16px; width: 100%; text-align: center; font-size: 10px; }}
  </style>
</head>
<body>

<h3>🎶 Piano Playback with Highlight & Sound</h3>
<div class="piano" id="piano"></div>

<script>
const notes = {note_json};

const pianoKeys = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"];

function createKey(pitch, octave) {{
  const isSharp = pitch.includes("#");
  const key = document.createElement("div");
  key.className = "key" + (isSharp ? " black" : " white");
  const id = pitch + octave;
  key.id = id;
  const label = document.createElement("div");
  label.className = "label";
  label.innerText = id;
  key.appendChild(label);
  return key;
}}

function drawPiano() {{
  const piano = document.getElementById("piano");
  for (let o = 3; o <= 5; o++) {{
    pianoKeys.forEach(p => {{
      const key = createKey(p, o);
      piano.appendChild(key);
    }});
  }}
}}

async function play() {{
  await Tone.start();
  const synth = new Tone.PolySynth().toDestination();

  notes.forEach(n => {{
    const delay = n.start_time * 1000;
    setTimeout(() => {{
      const el = document.getElementById(n.pitch);
      if (el) el.classList.add("active");
      synth.triggerAttackRelease(n.pitch, n.duration);
      setTimeout(() => {{
        if (el) el.classList.remove("active");
      }}, n.duration * 1000);
    }}, delay);
  }});
}}

drawPiano();
play();
</script>
</body>
</html>
'></iframe>
"""))
